In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [2]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=TotalRebounds&agg=Accumulated"

In [7]:
for year in years :
    url = url_data.format(year)
    data= requests.get(url)
    with open("rebounding_leaders/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [8]:
PATH="C:/Users/oki/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=PATH)

C:\Users\oki\AppData\Local\Temp\ipykernel_2460\3824445588.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


In [9]:
for year in years:
 url = url_data.format(year)
driver.get(url)  
driver.execute_script("window.scrollTo(1,50)")
time.sleep(2)
    
with open("rebounding_leaders/{}.html".format(year), "w+") as f:
    f.write(driver.page_source)
    


In [10]:
dfs = []
for year in years:
    with open("rebounding_leaders/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

In [12]:
players = pd.concat(dfs)
players

,Rank,Player,Team,GamesG,Total ReboundsTR,Ave.,Per 40 min.40m,Year
0,1,"TURKCAN, MIRSADTURKCAN, M.",CSKCSKA MoscowCSKA Moscow,20,209,10.45,16.33,2003
1,2,"SABONIS, ARVYDASSABONIS, A.",ZALZalgirisZalgiris,18,193,10.72,15.18,2003
2,3,"BASTON, MACEOBASTON, M.",MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv,21,170,8.10,13.59,2003
3,4,"BLAIR, JOSEPHBLAIR, J.",ULKUlkerUlker,17,163,9.59,11.19,2003
4,5,"BEARD, TANOKABEARD, T.",ZALZalgirisZalgiris,20,152,7.60,11.71,2003
...,...,...,...,...,...,...,...,...
45,45,"THOMAS, DESHAUNTHOMAS, D.",BAYFC Bayern MunichFC Bayern Munich,36,115,3.19,5.60,2021
46,47,"DA SILVA, OSCARDA SILVA, O.",BERALBA BerlinALBA Berlin,33,114,3.45,7.50,2021
47,48,"SMITS, ROLANDSSMITS, R.",BARFC BarcelonaFC Barcelona,36,112,3.11,8.37,2021
48,49,"PAPANIKOLAOU, KOSTASPAPANIKOLAOU, K.",OLYOlympiacos PiraeusOlympiacos Piraeus,37,111,3.00,5.12,2021


In [13]:
engine = create_engine('sqlite:///rebounding_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
players.to_sql(
    name = "rebounding_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

950

In [16]:
players_select = session.execute('select * from "rebounding_leaders"')
players_select.all()

[(0, 1, 'TURKCAN, MIRSADTURKCAN, M.', 'CSKCSKA MoscowCSKA Moscow', 20, 209, 10.45, 16.33, 2003),
 (1, 2, 'SABONIS, ARVYDASSABONIS, A.', 'ZALZalgirisZalgiris', 18, 193, 10.72, 15.18, 2003),
 (2, 3, 'BASTON, MACEOBASTON, M.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv', 21, 170, 8.1, 13.59, 2003),
 (3, 4, 'BLAIR, JOSEPHBLAIR, J.', 'ULKUlkerUlker', 17, 163, 9.59, 11.19, 2003),
 (4, 5, 'BEARD, TANOKABEARD, T.', 'ZALZalgirisZalgiris', 20, 152, 7.6, 11.71, 2003),
 (5, 6, 'MARCONATO, DENISMARCONATO, D.', 'TREBenetton BasketBenetton Basket', 19, 150, 7.89, 12.35, 2003),
 (6, 7, 'TOMASEVIC, DEJANTOMASEVIC, D.', 'VBCPamesa ValenciaPamesa Valencia', 17, 148, 8.71, 11.22, 2003),
 (7, 8, 'VUJCIC, NIKOLAVUJCIC, N.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv', 21, 145, 6.9, 7.91, 2003),
 (8, 9, 'DELFINO, CARLOSDELFINO, C.', 'FORSkipper Fortitudo BolognaSkipper Fortitudo Bologna', 21, 126, 6.0, 7.76, 2003),
 (9, 10, 'PARKER, ANTHONYPARKER, A.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel 